In [20]:
import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [21]:
df=pd.read_csv(r"C:\Users\dell\Desktop\mp\fer2013.csv")

In [22]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None


In [23]:
print(df.head())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training


In [25]:
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [26]:
num_labels = 7
epochs = 100
width, height = 48, 48

In [27]:
X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

In [28]:
train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

In [29]:
#X_train -= np.mean(X_train, axis=0)
#X_train /= np.std(X_train, axis=0)

#X_test -= np.mean(X_test, axis=0)
#X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [30]:
##designing the cnn
#1st convolution layer
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(32,kernel_size= (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))


In [31]:
#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.25))

In [32]:
#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

In [33]:
#fully connected neural networks
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(7))
model.add(Activation('softmax'))


In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 44, 44, 32)        9248      
_________________________________________________________________
batch_normalization_6 (Batch (None, 44, 44, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 18, 18, 64)        36928     
_________________________________________________________________
batch_normalization_7 (Batch (None, 18, 18, 64)       

In [35]:
#Compliling the model
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [36]:
hist = model.fit(X_train, train_y,
          batch_size=128,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)


#Saving the  model to  use it later on
model.save_weights("fer.h5")

Epoch 1/100
225/225 [==============================] - 292s 1s/step - loss: 2.1476 - accuracy: 0.1970 - val_loss: 1.7817 - val_accuracy: 0.2605
Epoch 2/100
225/225 [==============================] - 266s 1s/step - loss: 1.8620 - accuracy: 0.2374 - val_loss: 1.7973 - val_accuracy: 0.2561
Epoch 3/100
225/225 [==============================] - 265s 1s/step - loss: 1.6986 - accuracy: 0.3188 - val_loss: 1.6478 - val_accuracy: 0.3410
Epoch 4/100
225/225 [==============================] - 267s 1s/step - loss: 1.5516 - accuracy: 0.3828 - val_loss: 1.5442 - val_accuracy: 0.3845
Epoch 5/100
225/225 [==============================] - 266s 1s/step - loss: 1.4610 - accuracy: 0.4285 - val_loss: 1.6378 - val_accuracy: 0.3399
Epoch 6/100
225/225 [==============================] - 267s 1s/step - loss: 1.3722 - accuracy: 0.4698 - val_loss: 1.3912 - val_accuracy: 0.4575
Epoch 7/100
225/225 [==============================] - 277s 1s/step - loss: 1.3082 - accuracy: 0.4982 - val_loss: 1.3151 - val_accuracy:

225/225 [==============================] - 266s 1s/step - loss: 0.3969 - accuracy: 0.8607 - val_loss: 1.5032 - val_accuracy: 0.6127
Epoch 58/100
225/225 [==============================] - 266s 1s/step - loss: 0.3931 - accuracy: 0.8635 - val_loss: 1.4498 - val_accuracy: 0.6208
Epoch 59/100
225/225 [==============================] - 267s 1s/step - loss: 0.3830 - accuracy: 0.8659 - val_loss: 1.4575 - val_accuracy: 0.6088
Epoch 60/100
225/225 [==============================] - 266s 1s/step - loss: 0.3813 - accuracy: 0.8649 - val_loss: 1.4285 - val_accuracy: 0.6121
Epoch 61/100
225/225 [==============================] - 266s 1s/step - loss: 0.3832 - accuracy: 0.8683 - val_loss: 1.4655 - val_accuracy: 0.6113
Epoch 62/100
225/225 [==============================] - 267s 1s/step - loss: 0.3748 - accuracy: 0.8691 - val_loss: 1.4708 - val_accuracy: 0.6152
Epoch 63/100
225/225 [==============================] - 267s 1s/step - loss: 0.3694 - accuracy: 0.8712 - val_loss: 1.5206 - val_accuracy: 0.597

In [37]:
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
print('Training Accuracy: ', train_acc[-1])
print('Validation Accuracy: ', val_acc[-1])

# test loss and accuracy
score, acc = model.evaluate(X_test, test_y,
                            batch_size=32)
print('Test accuracy:', acc)

Training Accuracy:  0.9137552976608276
Validation Accuracy:  0.6132627725601196
113/113 [==============================] - 7s 64ms/step - loss: 1.6481 - accuracy: 0.6133
Test accuracy: 0.6132627725601196
